In [ ]:
import re

# Explanation of the core snippet, repeated for each tag:
# <rubric>\s*(\S.*?\S|\S)\s*</rubric>
#
# - \s* allows leading whitespace inside the tag before the real content
# - (\S.*?\S|\S) ensures there's at least one non-whitespace character:
#     - \S.*?\S means there's a non-whitespace at the start and at the end (with anything in-between).
#     - |\S covers the case of a single-character non-whitespace (so it doesn't force two).
# - \s* then allows trailing whitespace (e.g. <rubric>   content   </rubric>)
#
# We wrap these in  .*?  (non-greedy, DOTALL) to allow anything around/between the tags.

pattern = re.compile(
    r'^'
    r'.*?<rubric>\s*(\S.*?\S|\S)\s*</rubric>'
    r'.*?<eval>\s*(\S.*?\S|\S)\s*</eval>'
    r'.*?<answer>\s*(\S.*?\S|\S)\s*</answer>'
    r'.*?$',
    re.DOTALL
)

def validate_string_format(s: str) -> bool:
    """
    Returns True if 's' contains (in order):
      - <rubric> with at least one non-whitespace char
      - <eval> with at least one non-whitespace char
      - <answer> with at least one non-whitespace char
    and potentially anything before/between/after those blocks.
    Otherwise returns False.
    """
    return bool(pattern.match(s))


# ------------------------------------------------------------------------
# Example usage:

example_1 = """Some leading text
<rubric>   This is my rubric   </rubric>
random stuff ...
<eval> More content! </eval>   plus more text ...
<answer>  final answer here </answer>
and trailing text
"""

example_2 = """<rubric>   </rubric>  # no real content
<eval>some eval</eval>
<answer>some answer</answer>
"""

print("Example 1:", validate_string_format(example_1))  # EXPECT True (valid)
print("Example 2:", validate_string_format(example_2))  # EXPECT False (rubric block has only whitespace)


In [ ]:
import re

pattern = re.compile(
    r'^'
    # optional whitespace, then <rubric> with non-whitespace content
    r'\s*<rubric>\s*(\S(?:.*?\S)?)\s*</rubric>'
    # only whitespace allowed between </rubric> and <eval>
    r'\s*<eval>\s*(\S(?:.*?\S)?)\s*</eval>'
    # only whitespace allowed between </eval> and <answer>
    r'\s*<answer>\s*(\S(?:.*?\S)?)\s*</answer>\s*'
    r'$',
    re.DOTALL
)

def validate_string_format(s: str) -> bool:
    """
    Enforces this strict format:
      [optional whitespace]
      <rubric> (non-whitespace content) </rubric>
      [only whitespace allowed in between]
      <eval>   (non-whitespace content) </eval>
      [only whitespace allowed in between]
      <answer> (non-whitespace content) </answer>
      [optional whitespace]
      
    'Non-whitespace content' means at least one real character – not just spaces/tabs/newlines.
    Returns True if 's' matches exactly that structure; otherwise False.
    """
    return bool(pattern.match(s))

# ----------------------------------------------------------------
# EXAMPLES:

example_ok = """<rubric>  Some rubric content  </rubric>
<eval>   Multiple lines inside,
but not empty  </eval>
<answer>  final answer here </answer>"""

# example_bad_1 = """<rubric>Has content</rubric>\t<eval>Has content</eval>
# <answer>Has content</answer>"""
example_bad_1 = """Some leading text
<rubric>Has content</rubric>
# <eval>Has content</eval>
# <answer>Has content</answer>"""
# ❌ Disallowed because "Some leading text" is non-whitespace
#    outside the tags.

example_bad_2 = """<rubric>   </rubric>
<eval>Something</eval>
<answer>Something</answer>"""
# ❌ Disallowed because the <rubric> tag is empty (only spaces).

example_bad_3 = """<rubric>Rubric</rubric>
<eval>Eval</eval>  Some extra text
<answer>Answer</answer>"""
# ❌ Disallowed because "Some extra text" is non-whitespace
#    between </eval> and <answer>.

print("OK  ->", validate_string_format(example_ok))     # True
print("BAD1->", validate_string_format(example_bad_1))  # False
print("BAD2->", validate_string_format(example_bad_2))  # False
print("BAD3->", validate_string_format(example_bad_3))  # False


OK  -> True
BAD1-> True
BAD2-> False
BAD3-> False
